In [225]:
import pandas as pd
import numpy as np
import wikipedia
import re
from itertools import chain


In [136]:
df_dbpedia = pd.read_csv("./files/dbpedia_classes.csv")
df_imageNet = pd.read_csv("./files/ImageNet.csv")
df_imageNet['Class Name'] = df_imageNet['Class Name'].str.replace(", ",",")
df_imageNet['Class Name'] = df_imageNet['Class Name'].str.split(",")

### Query Wikipedia url and replace the url with dbpedia string

In [143]:
wikipedia_url_list = []
for class_list in df_imageNet['Class Name']:
    wikipedia_url_id = []
    for item in class_list:
        try:
            page_py = wikipedia.page(item)
            if page_py.url not in wikipedia_url_id:
                wikipedia_url_id.append(page_py.url)
        except:
            continue
    wikipedia_url_list.append(wikipedia_url_id)
    print(wikipedia_url_list)
        

PageError: Page id "Australasian robins" does not match any pages. Try another id!

In [90]:
for urls in wikipedia_url_list:
    for url in urls:
        print(url)

https://en.wikipedia.org/wiki/Trench
https://en.wikipedia.org/wiki/Tench
https://en.wikipedia.org/wiki/Goldfish
https://en.wikipedia.org/wiki/Great_white_shark
https://en.wikipedia.org/wiki/Shark_attack
https://en.wikipedia.org/wiki/Tiger_shark
https://en.wikipedia.org/wiki/Electric_ray
https://en.wikipedia.org/wiki/Narcinidae
https://en.wikipedia.org/wiki/Torpedo
https://en.wikipedia.org/wiki/Ostrich
https://en.wikipedia.org/wiki/Common_ostrich
https://en.wikipedia.org/wiki/Brambling
https://en.wikipedia.org/wiki/European_goldfinch
https://en.wikipedia.org/wiki/House_finch
https://en.wikipedia.org/wiki/Junko
https://en.wikipedia.org/wiki/Snowboard
https://en.wikipedia.org/wiki/Indigo_bunting
https://en.wikipedia.org/wiki/American_robin
https://en.wikipedia.org/wiki/Bulbul
https://en.wikipedia.org/wiki/Magpie
https://en.wikipedia.org/wiki/Chickadee
https://en.wikipedia.org/wiki/Dipper
https://en.wikipedia.org/wiki/Bald_eagle
https://en.wikipedia.org/wiki/Vulture
https://en.wikipedia.or

In [92]:
def wiki_to_db(wikipedia_url_list):
    dbpedia_url_list = []
    for urls in wikipedia_url_list:
        temp_list = []
        for url in urls:
            temp_list.append(url.replace('https://en.wikipedia.org/wiki/', 'http://dbpedia.org/resource/'))
        dbpedia_url_list.append(temp_list)
    return dbpedia_url_list

dbpedia_url_list = wiki_to_db(wikipedia_url_list)

In [97]:
for urls in dbpedia_url_list:
    for url in urls:
        print(url)

http://dbpedia.org/resource/Trench
http://dbpedia.org/resource/Tench
http://dbpedia.org/resource/Goldfish
http://dbpedia.org/resource/Great_white_shark
http://dbpedia.org/resource/Shark_attack
http://dbpedia.org/resource/Tiger_shark
http://dbpedia.org/resource/Electric_ray
http://dbpedia.org/resource/Narcinidae
http://dbpedia.org/resource/Torpedo
http://dbpedia.org/resource/Ostrich
http://dbpedia.org/resource/Common_ostrich
http://dbpedia.org/resource/Brambling
http://dbpedia.org/resource/European_goldfinch
http://dbpedia.org/resource/House_finch
http://dbpedia.org/resource/Junko
http://dbpedia.org/resource/Snowboard
http://dbpedia.org/resource/Indigo_bunting
http://dbpedia.org/resource/American_robin
http://dbpedia.org/resource/Bulbul
http://dbpedia.org/resource/Magpie
http://dbpedia.org/resource/Chickadee
http://dbpedia.org/resource/Dipper
http://dbpedia.org/resource/Bald_eagle
http://dbpedia.org/resource/Vulture
http://dbpedia.org/resource/Great_grey_owl
http://dbpedia.org/resource/

### Query the dbpedia Entities and get correponded dbpedia class

In [94]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper(
    "http://dbpedia.org/sparql"
)
sparql.setReturnFormat(JSON)
# gets the first 3 geological ages
# from a Geological Timescale database,
# via a SPARQL endpoint



In [199]:
def query_dbpedia(url):
    dbpedia_class_url_list= []
    dbpedia_class_name_list = []
    sparql.setQuery("""
    SELECT *
    WHERE
     {{
        <{var}>  rdf:type ?type . 
         FILTER ( CONTAINS ( str( ?type) , 'ontology' ) ) .

     }}
    """.format(var=url))
    
    try:
        ret = sparql.queryAndConvert()
        for r in ret["results"]["bindings"]:
            print(r['type']['value'])
            dbpedia_class_url_list.append(r['type']['value'])
            dbpedia_class_name_list.append(r['type']['value'].split('http://dbpedia.org/ontology/')[1])
            
    except Exception as e:
        print(e)
        dbpedia_class_url_list.append('')
    return dbpedia_class_url_list, dbpedia_class_name_list

def get_dbpedia_class(dbpedia_url_list):
    dbpedia_class_list = []
    dbpedia_class_name_list=[]
    for dbpedia_urls in dbpedia_url_list:
        temp_list = []
        temp_name_list=[]
        for url in dbpedia_urls:
            try:
                element_url, element_name = query_dbpedia(url)
                temp_list.append(element_url)
                temp_name_list.append(element_name)
            except:
                temp_list.append('Url do not exist')
                temp_name_list('class do not exist')
                continue

        dbpedia_class_list.append(temp_list)
        dbpedia_class_name_list.append(temp_name_list)
    return dbpedia_class_list, dbpedia_class_name_list

dbpedia_class_list,dbpedia_class_name_list = get_dbpedia_class(dbpedia_url_list)


http://dbpedia.org/ontology/Building
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Fish
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Fish
http://dbpedia.org/ontology/Disease
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Fish
http://dbpedia.org/ontology/Band
http://dbpedia.org/ontology/Band
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/MilitaryUnit
http://dbpedia.org/ontology/Weapon
http://dbpedia.org/ontology/Eukaryote
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Organisation
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Band
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/RecordLabel
http://dbpedia.org/ontology/Animal
http://dbpedia.org/onto

http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/RecordLabel
http://dbpedia.org/ontology/MythologicalFigure
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/FictionalCharacter
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/FictionalCharacter
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/FictionalCharacter
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
ht

http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Animal
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Animal
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Animal
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Place
http://dbpedia.org/ontology/Location
http://dbpedia.org/ontology/MusicalArtist
http://dbpedia.org/ontology/Country
http://dbpedia.org/ontology/PopulatedPlace
http://dbpedia.org/ontology/Grape
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Eukaryote
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/PersonFunction
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/EthnicGroup
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/EthnicGroup
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Mammal
http://dbpedia.org/ontology/Plant
http://dbpedia.org/

http://dbpedia.org/ontology/Person
http://dbpedia.org/ontology/Device
http://dbpedia.org/ontology/AmusementParkAttraction
http://dbpedia.org/ontology/RecordLabel
http://dbpedia.org/ontology/RoadJunction
http://dbpedia.org/ontology/MusicGenre
http://dbpedia.org/ontology/Automobile
http://dbpedia.org/ontology/Software
http://dbpedia.org/ontology/Ship
http://dbpedia.org/ontology/Device
http://dbpedia.org/ontology/PersonFunction
http://dbpedia.org/ontology/Agent
http://dbpedia.org/ontology/Company
http://dbpedia.org/ontology/Building
http://dbpedia.org/ontology/MusicalWork
http://dbpedia.org/ontology/Person
http://dbpedia.org/ontology/Person
http://dbpedia.org/ontology/Royalty
http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson
list index out of range
http://dbpedia.org/ontology/Agent
http://dbpedia.org/ontology/Building
http://dbpedia.org/ontology/Building
http://dbpedia.org/ontology/MusicGenre
http://dbpedia.org/ontology/Weapon
http://dbpedia.org/ontology/Place
http://dbp

http://dbpedia.org/ontology/Food
http://dbpedia.org/ontology/Settlement
http://dbpedia.org/ontology/Motorcycle
http://dbpedia.org/ontology/Organisation
http://dbpedia.org/ontology/Sport
http://dbpedia.org/ontology/HistoricPlace
http://dbpedia.org/ontology/RecordLabel
http://dbpedia.org/ontology/Building
http://dbpedia.org/ontology/MusicGenre
http://dbpedia.org/ontology/Device
http://dbpedia.org/ontology/Company
http://dbpedia.org/ontology/Device
http://dbpedia.org/ontology/Person
http://dbpedia.org/ontology/Artist
http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson
list index out of range
http://dbpedia.org/ontology/Place
http://dbpedia.org/ontology/Location
http://dbpedia.org/ontology/MusicalArtist
http://dbpedia.org/ontology/Country
http://dbpedia.org/ontology/PopulatedPlace
http://dbpedia.org/ontology/InformationAppliance
http://dbpedia.org/ontology/Place
http://dbpedia.org/ontology/PersonFunction
http://dbpedia.org/ontology/Album
http://dbpedia.org/ontology/Plant
ht

http://dbpedia.org/ontology/Organisation
http://dbpedia.org/ontology/Artwork
http://dbpedia.org/ontology/Automobile
http://dbpedia.org/ontology/Company
http://dbpedia.org/ontology/Drug
http://dbpedia.org/ontology/MusicGenre
http://dbpedia.org/ontology/Disease
http://dbpedia.org/ontology/MeanOfTransportation
http://dbpedia.org/ontology/MusicGenre
http://dbpedia.org/ontology/MeanOfTransportation
http://dbpedia.org/ontology/Ship
http://dbpedia.org/ontology/Building
http://dbpedia.org/ontology/Automobile
http://dbpedia.org/ontology/PersonFunction
http://dbpedia.org/ontology/Agent
http://dbpedia.org/ontology/Software
http://dbpedia.org/ontology/Artwork
http://dbpedia.org/ontology/MeanOfTransportation
http://dbpedia.org/ontology/MusicGenre
http://dbpedia.org/ontology/PersonFunction
http://dbpedia.org/ontology/Agent
http://dbpedia.org/ontology/Person
http://dbpedia.org/ontology/Device
http://dbpedia.org/ontology/Agent
http://dbpedia.org/ontology/Software
http://dbpedia.org/ontology/Bridge
htt

In [200]:
dbpedia_class_name_list

[[['Building'], []],
 [['Insect', 'Fish']],
 [['Insect', 'Fish'], ['Disease']],
 [['Insect', 'Fish']],
 [],
 [['Band'], ['Band', 'Insect'], ['MilitaryUnit', 'Weapon']],
 [],
 [],
 [],
 [['Eukaryote'], []],
 [['Bird']],
 [['Bird']],
 [['Bird']],
 [[], ['Organisation']],
 [['Bird']],
 [['Bird']],
 [['Insect']],
 [],
 [['Bird']],
 [['Band', 'Insect']],
 [['Insect']],
 [],
 [['Bird']],
 [['RecordLabel', 'Animal', 'Eukaryote', 'Species', 'Bird']],
 [[]],
 [[], ['Insect', 'Eukaryote']],
 [['Amphibian']],
 [],
 [['Amphibian']],
 [['Amphibian'], ['Insect', 'Amphibian']],
 [[], ['Insect', 'Amphibian']],
 [],
 [['Eukaryote'], ['Eukaryote'], [], ['Insect', 'Amphibian']],
 [['Reptile']],
 [['Reptile']],
 [['Insect', 'Reptile']],
 [['Eukaryote']],
 [['Reptile']],
 [[]],
 [['Reptile'], ['Insect', 'Reptile']],
 [[], ['Insect']],
 [['Eukaryote']],
 [['Reptile']],
 [[]],
 [[]],
 [['Reptile']],
 [['Reptile']],
 [['Reptile'], []],
 [['Eukaryote'], []],
 [['Reptile']],
 [['Reptile']],
 [['Reptile']],
 [['

In [240]:
df_imageNet['dbpedia_class_url'] = dbpedia_class_list
df_imageNet['dbpedia_class'] = dbpedia_class_name_list
df_imageNet['dbpedia_entity_url']= dbpedia_url_list
df_imageNet['wikipedia_url'] = wikipedia_url_list


ValueError: Length of values (15) does not match length of index (1000)

In [521]:
df_imageNet

,Class ID,Class Name,dbpedia_class_url,dbpedia_class,dbpedia_entity_url
0,0,"[tench, Tinca tinca]","[[http://dbpedia.org/ontology/Building], []]",[Building],"[http://dbpedia.org/resource/Trench, http://dbpedia.org/resource/Tench]"
1,1,"[goldfish, Carassius auratus]","[[http://dbpedia.org/ontology/Insect, http://dbpedia.org/ontology/Fish]]","[Insect, Fish]",[http://dbpedia.org/resource/Goldfish]
2,2,"[great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias', ]","[[http://dbpedia.org/ontology/Insect, http://dbpedia.org/ontology/Fish], [http://dbpedia.org/ontology/Disease]]","[Insect, Fish, Disease]","[http://dbpedia.org/resource/Great_white_shark, http://dbpedia.org/resource/Shark_attack]"
3,3,"[tiger shark, Galeocerdo cuvieri]","[[http://dbpedia.org/ontology/Insect, http://dbpedia.org/ontology/Fish]]","[Insect, Fish]",[http://dbpedia.org/resource/Tiger_shark]
4,4,"[hammerhead, hammerhead shark]",[],[],[]
...,...,...,...,...,...
995,995,[earthstar],[],[],[]
996,996,"[hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa]","[[http://dbpedia.org/ontology/Eukaryote, http://dbpedia.org/ontology/Fungus, http://dbpedia.org/ontology/Species]]","[Eukaryote, Fungus, Species]",[http://dbpedia.org/resource/Grifola_frondosa]
997,997,[bolete],[[]],[],[http://dbpedia.org/resource/Bolette]
998,998,"[ear, spike, capitulum]",[[http://dbpedia.org/ontology/Person]],[Person],[http://dbpedia.org/resource/Earl]


In [241]:
class_lst = [list(chain.from_iterable(j)) for j in dbpedia_class_name_list]
df_imageNet['dbpedia_class']=class_lst

In [154]:
pd.set_option('display.max_colwidth', 0)

# Evaluate the 

# Extract all ImageNet classes that has a class Mapping and evaluation

In [384]:
word_2_vec_top3_max = pd.read_csv('./evaluation/files/word_2_vec_top3_max.csv')
jaccard_top3=  pd.read_csv('./evaluation/files/jaccard_top3.csv')
jarow_winkler_top3=  pd.read_csv('./evaluation/files/jarow_winkler_top3.csv')
levenshtein_top3=  pd.read_csv('./evaluation/files/levenshtein_top3.csv')
df_baseline=df_imageNet.query("dbpedia_class.str.len() >0", engine="python")


In [400]:
df_word_2_vec_top3_max

,imageNet_class,mapped_class_1,mapped_class_2,mapped_class_3
0,"goldfish, Carassius auratus",fish,reptile,cat
1,"great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',",reptile,mammal,fish
2,"tiger shark, Galeocerdo cuvieri",reptile,mammal,fish
3,"hammerhead, hammerhead shark",reptile,mammal,fish
4,"electric ray, crampfish, numbfish, torpedo",solar eclipse,rocket,rocket engine
...,...,...,...,...
973,"stinkhorn, carrion fungus",fungus,insect,bacteria
974,"hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa",bird,cat,garden
975,bolete,grape,cycad,fern
976,"ear, spike, capitulum",storm surge,brain,bone


In [405]:
def read_and_process_mapping_data(file_path):
    df = pd.read_csv(file_path)
    df = df.T.rename_axis('imageNet_class').reset_index()
    df.drop(index=df.index[0], axis=0, inplace=True)
    df.reset_index(inplace = True,drop = True)
    df = df.rename(columns={0: "mapped_class_1", 1: "mapped_class_2", 2:"mapped_class_3"})
    return df
df_bert = read_and_process_mapping_data('mapping_results_bert.csv')
def read_and_process_mapping_data_1(file_path):
    df = pd.read_csv(file_path)
    df = df.T.rename_axis('imageNet_class').reset_index()
    df.reset_index(inplace = True,drop = True)
    df = df.rename(columns={0: "mapped_class_1", 1: "mapped_class_2", 2:"mapped_class_3"})
    return df
df_word_2_vec_top3_max = read_and_process_mapping_data_1('./evaluation/processed_files/word_2_vec_top3_max.csv')
df_word_2_vec_top3_mean = read_and_process_mapping_data_1('./evaluation/processed_files/word_2_vec_top3_mean.csv')
df_wiki_2_vec_top3_max = read_and_process_mapping_data_1('./evaluation/processed_files/wiki_2_vec_top3_max.csv')
df_wiki_2_vec_top3_mean = read_and_process_mapping_data_1('./evaluation/processed_files/wiki_2_vec_top3_mean.csv')
df_jaccard_top3 = read_and_process_mapping_data_1('./evaluation/files/jaccard_top3.csv')
df_jarow_winkler_top3 = read_and_process_mapping_data_1('./evaluation/files/jarow_winkler_top3.csv')
df_levenshtein_top3 = read_and_process_mapping_data_1('./evaluation/files/levenshtein_top3.csv')


In [531]:
df_word_2_vec_top3_max[:50]

,imageNet_class,mapped_class_1,mapped_class_2,mapped_class_3
0,"tench, Tinca tinca",fish,lake,species
1,"goldfish, Carassius auratus",fish,reptile,cat
2,"great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',",reptile,mammal,fish
3,"tiger shark, Galeocerdo cuvieri",reptile,mammal,fish
4,"hammerhead, hammerhead shark",reptile,mammal,fish
5,"electric ray, crampfish, numbfish, torpedo",solar eclipse,rocket,rocket engine
6,stingray,reptile,mammal,fish
7,cock,bird,hollywood cartoon,cat
8,hen,bird,cat,animal
9,"ostrich, Struthio camelus",animal,reptile,horse


In [408]:
def select_rows_and_process(df):
    df = df.loc[list(df_baseline.index)]
    df['val_dbpedia_class'] = df_baseline['dbpedia_class']
    temp = []
    y = []
    for item in df['val_dbpedia_class']:
        temp.append([x.lower() for x in item])
        y.append(0)
    df['val_dbpedia_class'] = temp
    df['y'] = y
    return df
df_bert_processed = select_rows_and_process(df_bert)
df_jaccard_top3_processed = select_rows_and_process(df_jaccard_top3)
df_jarow_winkler_top3_processed = select_rows_and_process(df_jarow_winkler_top3)
df_levenshtein_top3_processed = select_rows_and_process(df_levenshtein_top3)
df_word_2_vec_top3_max_processed =  select_rows_and_process(df_word_2_vec_top3_max)


In [433]:
def evaluate(df):
    for row in df.itertuples():
        if (row.mapped_class_1 in df.at[row.Index,'val_dbpedia_class']) or (row.mapped_class_2 in df.at[row.Index,'val_dbpedia_class']) or (row.mapped_class_3 in df.at[row.Index,'val_dbpedia_class']):
            df.at[row.Index,'y'] = 1
    score = df[df['y'] ==1].count()['y']/len(df)
    print(score)
    return score

score = evaluate(df_jaccard_top3_processed)
score

0.005738880918220947


0.005738880918220947

In [534]:
df_word_2_vec_top3_max_processed[50:100]

,imageNet_class,mapped_class_1,mapped_class_2,mapped_class_3,y_validation,label
67,"diamondback, diamondback rattlesnake, Crotalus adamanteus",reptile,mammal,bird,[eukaryote],0
68,"sidewinder, horned rattlesnake, Crotalus cerastes",reptile,arachnid,mammal,"[insect, eukaryote]",0
69,trilobite,mammal,eukaryote,crustacean,[band],0
70,"harvestman, daddy longlegs, Phalangium opilio",fish,family,crustacean,"[animal, arachnid, eukaryote, species]",0
73,"barn spider, Araneus cavaticus",horse,arachnid,insect,[arachnid],1
77,"wolf spider, hunting spider",arachnid,mammal,reptile,[insect],0
79,centipede,insect,arachnid,reptile,"[insect, animal]",1
80,black grouse,brown dwarf,bird,cat,[bird],1
81,ptarmigan,species,mammal,bird,[bird],1
82,"ruffed grouse, partridge, Bonasa umbellus",bird,species,conifer,"[bird, animal, eukaryote, species, bird]",1


In [437]:
def evaluate_print(df):
    for row in df.itertuples():
        if (row.mapped_class_1 in df.at[row.Index,'val_dbpedia_class']) or (row.mapped_class_2 in df.at[row.Index,'val_dbpedia_class']) or (row.mapped_class_3 in df.at[row.Index,'val_dbpedia_class']):
            df.at[row.Index,'y'] = 1
    print(df[df['y'] ==1])
evaluate_print(df_word_2_vec_top3_max_processed)


                                                                          imageNet_class  \
1    goldfish, Carassius auratus                                                           
2    great white shark, white shark, man-eater, man-eating shark, Carcharodon caharias',   
3    tiger shark, Galeocerdo cuvieri                                                       
11   goldfinch, Carduelis carduelis                                                        
12   house finch, linnet, Carpodacus mexicanus                                             
..                                         ...                                             
871  trimaran                                                                              
888  viaduct                                                                               
944  artichoke, globe artichoke                                                            
970  alp                                                                        

# Other Experiments

In [ ]:
wikidata_id_list = []
for class_list in df_imageNet['Class Name']:
    wikidata_id = []
    for item in class_list:
        try:
            page_py = wikipedia.page(item)
            for url in page_py.references:
                id_str = re.search("Q[0-9]{2,}", url)
                if id_str is not None:
                    print(id_str.group())

                    if id_str.group() not in wikidata_id:
                        wikidata_id.append(id_str.group())
                        print(wikidata_id)
        except:
            continue
    wikidata_id_list.append(wikidata_id)
    print(wikidata_id_list)
        

In [ ]:
df_imageNet['wikidata_id'] = wikidata_id_list

In [ ]:
wikipedia_revision_id_list = []
for class_list in df_imageNet['Class Name']:
    wikipedia_revision_id = []
    for item in class_list:
        try:
            page_py = wikipedia.page(item)
            if page_py.revision_id not in wikipedia_revision_id:
                wikipedia_revision_id.append(page_py.revision_id)
        except:
            continue
    wikipedia_revision_id_list.append(wikipedia_revision_id)
    print(wikipedia_revision_id_list)
        

In [55]:
def query_dbpedia(url):
    dbpedia_class_url_list= []
    sparql.setQuery("""
    SELECT *
    WHERE
     {{
        <{var}>  rdf:type ?type . 
         FILTER ( CONTAINS ( str( ?type) , 'ontology' ) ) .

     }}
    """.format(var=url))
    
    try:
        ret = sparql.queryAndConvert()
        for r in ret["results"]["bindings"]:
            print(r['type']['value'])
            dbpedia_class_url_list.append(r['type']['value'])
    except Exception as e:
        print(e)
        pass
    return dbpedia_class_url_list



for url in df_wiki_db['dbpedia_url']:
    print(url)
    if len(url) ==1:
        print(query_dbpedia(dbpedia_urls[2:-2]))
    else: 
        for u in url:
            print(query_dbpedia(u[2:-2]))


['http://dbpedia.org/resource/Tench']
[]
[]
[]
[]
[]
[]
[]
<urlopen error [Errno 61] Connection refused>
[]


KeyboardInterrupt: 

In [195]:
dbpedia_class_list

[[['http://dbpedia.org/ontology/Building'], []],
 [['http://dbpedia.org/ontology/Insect', 'http://dbpedia.org/ontology/Fish']],
 [['http://dbpedia.org/ontology/Insect', 'http://dbpedia.org/ontology/Fish'],
  ['http://dbpedia.org/ontology/Disease']],
 [['http://dbpedia.org/ontology/Insect', 'http://dbpedia.org/ontology/Fish']],
 [],
 [['http://dbpedia.org/ontology/Band'],
  ['http://dbpedia.org/ontology/Band', 'http://dbpedia.org/ontology/Insect'],
  ['http://dbpedia.org/ontology/MilitaryUnit',
   'http://dbpedia.org/ontology/Weapon']],
 [],
 [],
 [],
 [['http://dbpedia.org/ontology/Eukaryote'], []],
 [['http://dbpedia.org/ontology/Bird']],
 [['http://dbpedia.org/ontology/Bird']],
 [['http://dbpedia.org/ontology/Bird']],
 [[], ['http://dbpedia.org/ontology/Organisation']],
 [['http://dbpedia.org/ontology/Bird']],
 [['http://dbpedia.org/ontology/Bird']],
 [['http://dbpedia.org/ontology/Insect']],
 [],
 [['http://dbpedia.org/ontology/Bird']],
 [['http://dbpedia.org/ontology/Band', 'http:/

In [197]:
def extract_class_from_url():
    for urls in dbpedia_class_list:
        for url in urls:
            for u in url:
                print(u)
extract_class_from_url()

http://dbpedia.org/ontology/Building
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Fish
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Fish
http://dbpedia.org/ontology/Disease
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Fish
http://dbpedia.org/ontology/Band
http://dbpedia.org/ontology/Band
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/MilitaryUnit
http://dbpedia.org/ontology/Weapon
http://dbpedia.org/ontology/Eukaryote
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Organisation
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/Band
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Insect
http://dbpedia.org/ontology/Bird
http://dbpedia.org/ontology/RecordLabel
http://dbpedia.org/ontology/Animal
http://dbpedia.org/onto

In [131]:
try:
    page_py = wikipedia.page('orange',auto_suggest=False,preload=False)
except wikipedia.DisambiguationError as e:
    s = e.options[0]
    print(s)
    page_py = wikipedia.page(s)

Orange (fruit)
